<a href="https://colab.research.google.com/github/LXW4939/2019_demo_public/blob/master/read_csv_from_local_device.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/LXW4939/2019_demo_public/blob/master/data_per200.csv

In [0]:
!ls

点击左边的[代码段] 输出 uploading ， 找到open files from your local file system ,选择插入，即可插入下方代码段

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

点击[run]， 选择本地的文件即可上传成功，此时若直接使用pd.read_csv()可能会报错，进过测试可使用df = pd.read_csv(io.StringIO(uploaded['data_per200.csv'].decode('utf-8')))进行读取csv文件

In [0]:
print (uploaded['data_per200.csv'][:10].decode('utf-8') + '...')

In [0]:
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded['data_per200.csv'].decode('utf-8')))
df.head()

In [0]:
data = pd.read_csv('data_per200.csv')

In [0]:
data.head()

In [0]:
from sklearn import datasets
import numpy as np
import sklearn
import time
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

In [0]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
# Import data
start = time.time()
data = pd.read_csv('./data_per200.csv')
# Extract feature and target np arrays (inputs for placeholders)
input_x = data.iloc[:, 0:-1]
input_y = data.iloc[:, -1]
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_label, y_test_label = train_test_split(input_x, input_y, test_size = 0.25, random_state = 0)
class_name = list(set(y_train_label))
class_nums = len(class_name)
class_dict={}
for idx, name in enumerate(class_name):
    class_dict[name] = idx
y_train = y_train_label.map(class_dict)
y_test = y_test_label.map(class_dict)

# Parameters
num_steps = 2000  # Total steps to train
num_classes = len(set(input_y))
num_features = len(data.columns) - 1
num_trees = 500
max_nodes = 1000

# Input and Target placeholders
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.int64, shape=[None])
# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes, num_features=num_features, num_trees=num_trees, max_nodes=max_nodes).fill()
# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(), resources.initialize_resources(resources.shared_resources()))
# Start TensorFlow session
sess = tf.Session()
# Run the initializer
sess.run(init_vars)
# Training
for i in range(1, num_steps + 1):
    _, l = sess.run([train_op, loss_op], feed_dict={X: X_train, Y: y_train})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: X_train, Y: y_train})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))
# Test Model
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: X_test, Y: y_test}))
end = time.time()
print("time: ", end - start)